In [ ]:
cd ../../

In [ ]:
from src.models.resnet.resnet import ResNet18
import torch
from src.pruning.slth.edgepopup_ensemble_output import modify_module_for_slth
from src.data.cifar.cifar10 import get_cifar10_data_loaders
from src.utils.seed import torch_fix_seed
device = "cuda"
batch_size = 128
seed = 2

if seed == 0:
    torch_fix_seed(0)
    base_path = "./logs/CIFAR10/is_prune/ensemble_output/seed_0/2024_03_27_16_44_39/resnet_slth{}_state.pkl"
    no_prune_path = "./logs/CIFAR10/no_prune/seed_0/2024_03_28_15_30_04/model_state.pkl"

if seed == 1:
    torch_fix_seed(1)
    base_path = "./logs/CIFAR10/is_prune/ensemble_output/seed_1/2024_03_27_16_44_39/resnet_slth{}_state.pkl"
    no_prune_path = "./logs/CIFAR10/no_prune/seed_1/2024_03_28_15_30_04/model_state.pkl"

if seed == 2:
    torch_fix_seed(2)
    base_path = "./logs/CIFAR10/is_prune/ensemble_output/seed_2/2024_03_27_16_44_39/resnet_slth{}_state.pkl"
    no_prune_path = "./logs/CIFAR10/no_prune/seed_2/2024_03_28_15_30_04/model_state.pkl"

In [ ]:
train_loader, test_loader = get_cifar10_data_loaders(batch_size)

import numpy as np
from tqdm import tqdm
import torch.nn.functional as F
import torch.nn as nn

device = 'cuda'

paths = {"slth{}_path".format(str(i)): base_path.format(str(i)) for i in np.arange(1, 7)}
weights = {"slth{}_weight".format(str(i)): torch.load(base_path.format(str(i))) for i in np.arange(1, 7)}


In [ ]:
# すべてのweightsをリストに格納
import copy
from itertools import islice

weight_list = dict(islice(weights.items(), 1, 3)).values()
original = weights['slth1_weight']
copied = copy.deepcopy(original)

# slth1_weightの'score'を含むキーのみを処理
for key in copied:
    if 'scores' in key:
        total_sum = copied[key].clone()  # slth1_weightの値を基点とする
        for weight in weight_list:
            total_sum += weight[key]
        # 合計した値をweightsの数で割る
        copied[key] = total_sum / len(weight_list)


In [ ]:
for key in copied:
    if 'scores' in key:
        print(torch._is_all_true(original[key] == copied[key]).item())

In [ ]:
model = modify_module_for_slth(ResNet18(), 0.3).to("cuda")
model.load_state_dict(copied)
model.eval()

In [ ]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        _, predicted = torch.max(model(images), 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
    acc = 100 * correct / total
    print(acc)

In [ ]:
model(images)